## Guided Generation Test
Original code is from here: https://github.com/dottxt-ai/outlines

There were some questionable things in this code, so I worked with Google Gemini
to clean and explain the code a little better.

According to Gemini the code was also out-of-date...

# %%capture suppresses the long output 

Also, fun fact, things like %%capture must be the very first line in a cell... 

In [1]:
%%capture
!pip install outlines

In [2]:
%%capture
!pip install --upgrade transformers accelerate torch

In [3]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 21.4 MB/s eta 0:00:0000:0100:01m


In [4]:
#imports
import outlines
import torch
from enum import Enum
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from typing import List


In [5]:
# ---------------------------------------------------------
# 1. SETUP: DEFINING THE "SHAPE" OF THE DATA
# ---------------------------------------------------------
# We aren't just writing a Python class here. Outlines will look at this 
# Pydantic model and convert it into a complex "Grammar" or Regex 
# behind the scenes. This acts like a stencil for the AI.
class TicketPriority(str, Enum):
    low = "low"
    medium = "medium"
    high = "high"
    urgent = "urgent"

class ServiceTicket(BaseModel):
    priority: TicketPriority
    category: str
    requires_manager: bool
    summary: str
    action_items: List[str]

I was crashing Kaggle with this popup: Kernel Restarting
The kernel for notebook_source.ipynb appears to have died. It will restart automatically.

So we switched to a 4 bit quantitized model that is smaller, to try this again...

In [ ]:
# ---------------------------------------------------------
# 2. SETUP: LOADING THE BRAINS
# ---------------------------------------------------------
MODEL_NAME = "microsoft/Phi-3-mini-4k-instruct"

print(f"\nLoading {MODEL_NAME}...")

# This config loads the model in 4-bit mode. 
# It creates a tiny memory footprint so the kernel won't crash.
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

llm = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="cuda",
    quantization_config=quantization_config,
    
    # CRITICAL FIX 1: Use native code (Avoids DynamicCache error)
    trust_remote_code=False, 
    
    # CRITICAL FIX 2: Force standard attention (Prevents T4 Crash)
    attn_implementation="eager" 
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Create the Outlines wrapper
# outlines.from_transformers wraps the standard LLM. 
# It intercepts the model's generation process. When the model tries to 
# pick the next word, Outlines checks the Pydantic schema (ServiceTicket)
# and bans any token that doesn't fit the JSON structure.
wrapped_model = outlines.from_transformers(llm, tokenizer)


Loading microsoft/Phi-3-mini-4k-instruct...


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

In [ ]:
# ---------------------------------------------------------
# 3. THE INPUT
# ---------------------------------------------------------
customer_email = """
Subject: URGENT - Cannot access my account after payment
I paid for the premium plan 3 hours ago and still can't access any features.
Please fix this immediately or refund my payment.
"""

# We format the prompt using Phi-3's specific chat template tags
# (<|im_start|>, etc).
prompt = f"""<|im_start|>user
Analyze this customer email:
{customer_email}
<|im_end|>
<|im_start|>assistant
"""

# ---------------------------------------------------------
# 4. EXECUTION: GENERATING THE TICKET
# ---------------------------------------------------------
# Here is the fix for the confusing variable usage.
# When we pass `ServiceTicket` as the second argument, Outlines does three things:
# 1. Forces the LLM to output valid JSON matching the ServiceTicket schema.
# 2. Waits for generation to finish.
# 3. AUTOMATICALLY converts that JSON string into a Python 'ServiceTicket' object.
structured_ticket_object = model(
    prompt,
    ServiceTicket,
    max_new_tokens=500
)

# ---------------------------------------------------------
# 5. LOGIC: USING THE RESULT
# ---------------------------------------------------------
# Because 'structured_ticket_object' is already a Python object, 
# we can access properties using dot notation (.priority) immediately.
# No JSON parsing is required here.

def alert_manager(ticket):
    print(f"ALARM: {ticket.priority.upper()} priority issue: {ticket.summary}")

# We check the Enums and Booleans directly
if structured_ticket_object.priority == TicketPriority.urgent or structured_ticket_object.requires_manager:
    alert_manager(structured_ticket_object)
else:
    print("Ticket routed to standard queue.")